In [1]:
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification, create_optimizer
from sklearn.model_selection import train_test_split
from tensorflow.data import Dataset
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import shutil

E0000 00:00:1721820799.042290      77 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
D0724 11:33:19.050994726      77 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D0724 11:33:19.051010129      77 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D0724 11:33:19.051013334      77 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D0724 11:33:19.051015647      77 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D0724 11:33:19.051018042      77 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defa

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Ініціалізація TPU
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1721820833.177510      77 service.cc:145] XLA service 0x574b8f51c770 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1721820833.177567      77 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1721820833.177572      77 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1721820833.177575      77 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1721820833.177578      77 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1721820833.177580      77 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1721820833.177583      77 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1721820833.177586      77 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1721820833.177608      77 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:6, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


In [3]:
# Завантаження датасету
file_path = '/kaggle/input/toxic-dataset/train_data_11000.csv'  # Update the path to your dataset
data = pd.read_csv(file_path)

# Попередня обробка датасету
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['comment_text'].tolist(), 
    data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values.tolist(), 
    test_size=0.2
)

# Токенізатор
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Функція токенізації текстів
def tokenize_texts(texts, labels, tokenizer, max_length=512):
    input_ids = []
    attention_masks = []
    for text in texts:
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            truncation=True
        )
        input_ids.append(encoding['input_ids'])
        attention_masks.append(encoding['attention_mask'])

    return (
        np.array(input_ids),
        np.array(attention_masks),
        np.array(labels)
    )

# Токенізація датасетів
train_input_ids, train_attention_masks, train_labels = tokenize_texts(train_texts, train_labels, tokenizer)
val_input_ids, val_attention_masks, val_labels = tokenize_texts(val_texts, val_labels, tokenizer)

# Функція створення TensorFlow датасетів
def create_tf_dataset(input_ids, attention_masks, labels, batch_size=32):
    dataset = Dataset.from_tensor_slices(({'input_ids': input_ids, 'attention_mask': attention_masks}, labels))
    dataset = dataset.shuffle(buffer_size=len(input_ids)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

BATCH_SIZE = 32

train_dataset = create_tf_dataset(train_input_ids, train_attention_masks, train_labels, BATCH_SIZE)
val_dataset = create_tf_dataset(val_input_ids, val_attention_masks, val_labels, BATCH_SIZE)

In [4]:
with strategy.scope():
    # Завантаження базової моделі BERT для класифікації
    model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)
    
    # Change the classifier layer to include sigmoid activation
    model.classifier.activation = tf.keras.activations.sigmoid
    
    # Параметри тренування
    EPOCHS = 3
    steps_per_epoch = len(train_texts) // BATCH_SIZE
    num_train_steps = steps_per_epoch * EPOCHS
    num_warmup_steps = int(num_train_steps * 0.1)

    # Оптимізатор та функція втрат
    optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=num_warmup_steps, num_train_steps=num_train_steps)
    loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)

    # Компіляція моделі
    model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

    # Виведення архітектури моделі
    model.summary()

I0000 00:00:1721820858.647116      77 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


All PyTorch model weights were used when initializing TFBertForSequenceClassification.



Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification"


_________________________________________________________________


...

 classifier (Dense)          multiple                  4614      


Total params: 109486854 (417.66 MB)


Trainable params: 109486854 (417.66 MB)


Non-trainable params: 0 (0.00 Byte)


_________________________________________________________________


In [5]:
# Suppress AutoGraph warnings
@tf.autograph.experimental.do_not_convert
def train_model():
    with strategy.scope():
        # Training
        history = model.fit(train_dataset, epochs=EPOCHS, validation_data=val_dataset)

        # Evaluate the model
        val_loss, val_acc = model.evaluate(val_dataset)
        print(f'Validation loss {val_loss} accuracy {val_acc}')



train_model()

Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2024-07-24 11:35:39.054517: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


...

65/69 [===========================>..] - ETA: 0s - loss: 0.0578 - accuracy: 0.9947

67/69 [============================>.] - ETA: 0s - loss: 0.0574 - accuracy: 0.9949

69/69 [==============================] - ETA: 0s - loss: 0.0581 - accuracy: 0.9950

69/69 [==============================] - 3s 34ms/step - loss: 0.0581 - accuracy: 0.9950


Validation loss 0.058126989752054214 accuracy 0.9950000047683716


In [6]:
# Function to predict on a new comment
def predict_comment(comment, model, tokenizer):
    encoding = tokenizer.encode_plus(
        comment,
        add_special_tokens=True,
        max_length=512,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='tf',
        truncation=True
    )
    
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']
    
    preds = model.predict({'input_ids': input_ids, 'attention_mask': attention_mask})[0]
    
    if preds.shape[0] == 1:
        preds = preds[0]  # Get the first element if it's a batch

    return {
        'toxic': preds[0],
        'severe_toxic': preds[1],
        'obscene': preds[2],
        'threat': preds[3],
        'insult': preds[4],
        'identity_hate': preds[5]
    }


In [7]:

model_save_path = "/kaggle/working/model_save"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)


('/kaggle/working/model_save/tokenizer_config.json',
 '/kaggle/working/model_save/special_tokens_map.json',
 '/kaggle/working/model_save/vocab.txt',
 '/kaggle/working/model_save/added_tokens.json')

In [8]:
# Example usage
comment = "many think that he is a piece of shit and behaves like a bitch. but he's actually a nice guy"
prediction = predict_comment(comment, model, tokenizer)
print(prediction)

2024-07-24 11:38:46.098376: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


I0000 00:00:1721821126.728804     886 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(645ea50a79aef1df:0:0), session_name()


1/1 [==============================] - ETA: 0s

1/1 [==============================] - 13s 13s/step


{'toxic': 0.9442891, 'severe_toxic': 0.22260273, 'obscene': 0.8642334, 'threat': 0.122659326, 'insult': 0.81335664, 'identity_hate': 0.21014243}


I0000 00:00:1721821129.965091     886 tpu_compile_op_common.cc:245] Compilation of 645ea50a79aef1df:0:0 with session name  took 3.236249848s and succeeded
I0000 00:00:1721821129.985881     886 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(645ea50a79aef1df:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_predict_function_1062005084767875851", property.function_library_fingerprint = 14746271935209418884, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "2,512,;2,512,;", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1721821129.985907     886 tpu_compilation_cache_interface.cc:541] After adding entry for key 645ea50a7

In [9]:
from sklearn.metrics import classification_report
# Generate predictions for the validation set and compute classification report

def evaluate_model(model, val_dataset, val_labels):
    # Отримання прогнозів для валідаційного набору
    val_predictions = model.predict(val_dataset)
    
    # Перетворення прогнозів у формат NumPy
    predictions = np.array(val_predictions.logits).squeeze()

    # Бінаризація прогнозів
    predictions = np.where(predictions > 0.5, 1, 0)

    # Перетворення міток у формат NumPy
    true_labels = np.array(val_labels)

    # Виведення класифікаційного звіту
    report = classification_report(true_labels, predictions, target_names=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], zero_division=0)
    print(report)

# Виклик функції для оцінки моделі
evaluate_model(model, val_dataset, val_labels)

I0000 00:00:1721821141.485797     866 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(62be21a2d634c916:0:0), session_name()


 1/69 [..............................] - ETA: 17:02

 3/69 [>.............................] - ETA: 2s   

I0000 00:00:1721821145.158277     866 tpu_compile_op_common.cc:245] Compilation of 62be21a2d634c916:0:0 with session name  took 3.672443126s and succeeded
I0000 00:00:1721821145.179419     866 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(62be21a2d634c916:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_predict_function_15523122636136593976", property.function_library_fingerprint = 7895517189088660776, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "4,512,;4,512,;4,6,;", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1721821145.179457     866 tpu_compilation_cache_interface.cc:541] After adding entry for key 62be

 5/69 [=>............................] - ETA: 2s

...

I0000 00:00:1721821147.549128     875 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(a3d88ea5c4bdc227:0:0), session_name()


69/69 [==============================] - ETA: 0s

69/69 [==============================] - 21s 91ms/step


               precision    recall  f1-score   support

        toxic       0.05      0.05      0.05       189
 severe_toxic       0.00      0.00      0.00        17
      obscene       0.02      0.02      0.02       113
       threat       0.00      0.00      0.00         8
       insult       0.01      0.01      0.01       100
identity_hate       0.00      0.00      0.00        20

    micro avg       0.03      0.03      0.03       447
    macro avg       0.01      0.01      0.01       447
 weighted avg       0.03      0.03      0.03       447
  samples avg       0.00      0.00      0.00       447



I0000 00:00:1721821151.367600     875 tpu_compile_op_common.cc:245] Compilation of a3d88ea5c4bdc227:0:0 with session name  took 3.818423693s and succeeded
I0000 00:00:1721821151.394636     875 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(a3d88ea5c4bdc227:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_predict_function_15523122636136593976", property.function_library_fingerprint = 7895517189088660776, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "3,512,;3,512,;3,6,;", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1721821151.394670     875 tpu_compilation_cache_interface.cc:541] After adding entry for key a3d8